In [2]:
from pyspark.sql import SparkSession
import schemas
from test import most_used_tags


def init_spark():
    spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
    sc = spark.sparkContext
    return spark, sc


def main():
    spark, sc = init_spark()
    timestamp_format = 'YYYY-MM-DD HH:mm:ss z'
    dummy_switch = '-1.4.0-2018-12-22'
    # dummy_switch = '_dummy'

    projects_path = f'data/projects-1.4.0-2018-12-22.csv'
    projects = spark.read.csv(projects_path, header=True, timestampFormat=timestamp_format,
                          schema=schemas.projects)
    most_used_tags(projects)
    # nums = sc.parallelize([1, 2, 3, 4])
    # print(nums.map(lambda x: x * x).collect())
    # dist_file = sc.textFile('Libraries.io-open-data-1.4.0.tar.gz', 5000)
    # size = dist_file.map(lambda s: len(s)).reduce(lambda a, b: a+b)
    # print(size)


if __name__ == '__main__':
    main()



Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.Abstract

In [ ]:
spark, sc = init_spark()
import pyspark.sql.functions as F
dependencies_path = f'data/dependencies-1.4.0-2018-12-22.csv'
projects_path = f'data/projects-1.4.0-2018-12-22.csv'
timestamp_format = 'YYYY-MM-DD HH:mm:ss z'

dependencies = spark.read.csv(projects_path, header=True, schema=schemas.projects)
dependencies = spark.read.csv(dependencies_path, header=True, schema=schemas.dependencies).drop('ID').select('Project ID', 'Dependency Project ID')
dependency_pairs = dependencies.where(F.col('Dependency Project ID').isNotNull()).limit(10)
pageranks = dependency_pairs \
    .select('Project ID') \
    .union(dependency_pairs
           .select('Dependency Project ID')) \
    .distinct().alias('ID') \
    .withColumn('pagerank', F.lit(1))
dependency_counts = dependency_pairs.groupBy('Project ID').count()
# print(pageranks.count(), pageranks.take(5))
for it in range(2):
    print(f'It {it}')
    dependency_pairs = pageranks \
        .join(dependency_pairs, 'Project ID') \
        .join(dependency_counts, 'Project ID') \
        .withColumn('vote', F.col('pagerank') / F.col('count'))\
        .drop('pagerank', 'count')
    vote_sums = dependency_pairs\
        .groupBy('Dependency Project ID')\
        .sum('vote')\
        .withColumnRenamed('sum(vote)', 'vote_sum')
    # print(vote_sums.take(5))
    pageranks = pageranks\
        .join(vote_sums, pageranks['Project ID'] == vote_sums['Dependency Project ID'], how='left')\
        .drop('pagerank')\
        .drop('Dependency Project ID')\
        .withColumn('pagerank', 1 + F.col('vote_sum'))\
        .drop('vote_sum') \
        .fillna({'pagerank': 1})
    pageranks.show(10)
pageranks.join(projects.select('ID', 'SourceRank', pageranks['Project ID']==projects['ID'])).drop('ID').show(10)